<a href="https://colab.research.google.com/github/jack978397/Colab-AI-Transcribe-Translate/blob/main/1_Whisper_Transcription_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. 修正安裝指令 (加入 Google AI 庫) ---
!pip install openai-whisper yt-dlp srt "moviepy<2.0.0" Gradio google-generativeai
!apt-get install -y ffmpeg

print("\n--- API 環境安裝完成！ ---")


In [ ]:
import whisper
import torch
import os
import re
import datetime
import srt
import yt_dlp
import gc

# --- 1. 核心工具函式 ---
def format_srt_time(seconds):
    delta = datetime.timedelta(seconds=max(0, seconds))
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = delta.microseconds // 1000
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

def refine_srt_segments(segments, max_chars=18):
    if not segments: return []
    pre_filtered = []
    last_text = None
    for seg in segments:
        text = seg['text'].strip()
        if not text: continue
        if text == last_text and (seg['end'] - seg['start']) < 0.8:
            if pre_filtered: pre_filtered[-1]['end'] = seg['end']
            continue
        pre_filtered.append(seg)
        last_text = text

    final_segments = []
    punctuation_pattern = r'([，。、？！,.\!\?])'
    for segment in pre_filtered:
        text = segment['text'].strip()
        if not text or len(text) <= max_chars:
            final_segments.append(segment)
            continue
        start_time, end_time = segment['start'], segment['end']
        duration = end_time - start_time
        parts = re.split(punctuation_pattern, text)
        sub_sentences = [p for p in parts if p.strip()]
        if not sub_sentences:
            final_segments.append(segment)
            continue
        current_sentence = ""
        total_len = sum(len(s) for s in sub_sentences)
        if total_len <= 0: total_len = 1
        current_start = start_time
        for sub in sub_sentences:
            if len(current_sentence) > 0 and len(current_sentence + sub) > max_chars:
                ratio = len(current_sentence) / total_len
                new_end = current_start + (ratio * duration)
                final_segments.append({'start': current_start, 'end': new_end, 'text': current_sentence})
                current_start = new_end
                current_sentence = sub
            else:
                current_sentence += (" " if current_sentence and not re.match(r'^[，。、？！]', sub) else "") + sub
        if current_sentence:
            final_segments.append({'start': current_start, 'end': end_time, 'text': current_sentence})

    deduped = []
    for s in final_segments:
        if not deduped or s['text'] != deduped[-1]['text']:
            deduped.append(s)
        else:
            deduped[-1]['end'] = s['end']
    for i, seg in enumerate(deduped): seg['id'] = i
    return deduped

def generate_srt_from_segments(segments):
    srt_content = []
    for i, segment in enumerate(segments):
        start_time, end_time = format_srt_time(segment['start']), format_srt_time(segment['end'])
        text = segment['text'].strip()
        if text:
            srt_content.append(str(i + 1))
            srt_content.append(f"{start_time} --> {end_time}")
            srt_content.append(text)
            srt_content.append("")
    return "\n".join(srt_content)

def fix_and_process_srt(srt_content, remove_punctuation=True):
    try:
        cleaned_content = "\n".join(line for line in srt_content.strip().splitlines() if line.strip())
        subs = list(srt.parse(cleaned_content))
        for sub in subs:
            mod = sub.content.strip()
            if remove_punctuation: mod = mod.rstrip('.,。?!！?？')
            sub.content = mod.strip()
        return srt.compose(subs)
    except: return srt_content

# --- 2. 引擎模組 ---
class WhisperEngineCloud:
    def __init__(self):
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_size = None

    def load_model(self, model_size="large"):
        if self.model is not None and self.model_size == model_size: return
        self.model = whisper.load_model(model_size, device=self.device)
        self.model_size = model_size

    def transcribe(self, audio_path, prompt=""):
        return self.model.transcribe(audio_path, initial_prompt=prompt, verbose=True)

# 建立全域實例 (UI 會呼叫它)
engine = WhisperEngineCloud()
print("✅ [Cell 2] 核心邏輯載入完畢")


In [ ]:
import gradio as gr
import gc
import torch
import os
import yt_dlp
import traceback

# --- 核心批次處理邏輯 ---
def batch_process_transcription(files, yt_input, input_type, model_size, remove_punc, progress=gr.Progress()):
    if 'engine' not in globals():
        yield "❌ 錯誤：請先執行 Cell 2！", []
        return

    try:
        log_content = "🚀 啟動批次轉錄任務...\n"
        output_files = []

        # 決定處理列表
        if input_type == "YouTube 連結":
            if not yt_input:
                yield "❌ 錯誤：請貼上網址", []
                return
            target_files = [("YouTube", yt_input)]
        else:
            if not files:
                yield "❌ 錯誤：請先上傳檔案", []
                return
            target_files = [("Local", f.name) for f in files]

        for i, (f_type, f_path) in enumerate(target_files):
            filename = os.path.basename(f_path) if f_type == "Local" else "YouTube 影片"
            log_content += f"正在處理 ({i+1}/{len(target_files)}): {filename}...\n"
            yield log_content, output_files

            # --- 下載/準備音訊 ---
            if f_type == "YouTube":
                if os.path.exists('temp_audio.m4a'): os.remove('temp_audio.m4a')
                ydl_opts = {'format': 'm4a/bestaudio/best', 'outtmpl': 'temp_audio.%(ext)s', 'quiet': True, 'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'}
                with yt_dlp.YoutubeDL(ydl_opts) as ydl: ydl.download([f_path])
                audio_source = 'temp_audio.m4a'
            else:
                audio_source = f_path

            # --- AI 辨識 ---
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

            engine.load_model(model_size)
            result = engine.transcribe(audio_source)

            # --- 格式優化 ---
            refined = refine_srt_segments(result['segments'])
            srt_content = generate_srt_from_segments(refined)
            final_srt = fix_and_process_srt(srt_content, remove_punctuation=remove_punc)

            # --- 存檔 ---
            safe_name = "".join([c if c.isalnum() else "_" for c in filename])
            out_path = os.path.abspath(f"transcript_{safe_name}.srt")
            with open(out_path, "w", encoding="utf-8") as f: f.write(final_srt)

            output_files.append(out_path)
            log_content += f"✅ {filename} 轉錄完成！\n"
            yield log_content, output_files

        log_content += "\n--- 🎉 全部檔案辨識完畢 ---"
        yield log_content, output_files

    except Exception as e:
        traceback.print_exc()
        yield f"❌ 轉錄失敗：{str(e)}", output_files

# --- UI 佈置 ---
with gr.Blocks(title="Whisper Pro Batch") as demo:
    gr.Markdown("# 🚀 Whisper Pro 影音批次轉錄中心")

    with gr.Row():
        with gr.Column(scale=1):
            t_mode = gr.Radio(["檔案上傳 (支援多選)", "YouTube 連結"], label="輸入模式", value="檔案上傳 (支援多選)")
            t_files = gr.File(label="上傳區", file_count="multiple", visible=True)
            t_url = gr.Textbox(label="YouTube 網址", visible=False)
            t_model = gr.Dropdown(["base", "medium", "large"], label="使用模型", value="large")
            t_punc = gr.Checkbox(label="自動移除句尾標點", value=True)
            t_btn = gr.Button("🔥 啟動批次辨識", variant="primary")

        with gr.Column(scale=2):
            gr.Markdown("### 📋 處理日誌與成果下載")
            t_log = gr.Textbox(label="即時處理進度", lines=12, interactive=False)
            t_out = gr.File(label="📥 轉錄成果下載 (SRT 檔案列表)", file_count="multiple")

    t_mode.change(lambda x: (gr.update(visible=x=="檔案上傳 (支援多選)"), gr.update(visible=x=="YouTube 連結")), t_mode, [t_files, t_url])
    t_btn.click(batch_process_transcription, [t_files, t_url, t_mode, t_model, t_punc], [t_log, t_out])

demo.queue().launch(debug=True, share=True)
